# Modelo de Producción, Períodos múltiples

Acme Manufacturing Company firmó un contrato para entregar 180, 250, 190, 140, 220 y 250 ventanas para casa durante los siguientes seis meses. El costo de producción (mano de obra, material y servicios) por ventana varía por período y se estima que será de $50, $45, $55, $52, $48 y $50 durante los próximos seis meses. 

Para aprovechar las fluctuaciones del costo de fabricación, Acme puede producir más ventanas de las necesarias en un mes dado y conservar las unidades adicionales para entregarlas en meses posteriores. Esto supondrá un costo de almacenamiento del inventario a fin de mes, a razón de $8 por ventana por mes en los meses 1, 5 y 6. En los meses 2, 3 y 4, los costos por almacenaje son de $10 por ventana debido a la temporada de alta demanda.

Asumiendo que la empresa tiene una capacidad máxima de producción de 225 ventanas cada mes, desarrolle un programa lineal para determinar el programa de producción óptimo.

![Representación esquemática del sistema de producción e inventario](../images/ejemplo_p2.png)

## Inciso a

Formular el problema de programación lineal.

In [10]:
using Pkg
Pkg.instantiate()

In [11]:

Pkg.add("PrettyTables")


   Resolving package versions...
  No Changes to `C:\Users\Cindy\Documents\GitHub\labs-ms\Labs\lab1\Project.toml`
  No Changes to `C:\Users\Cindy\Documents\GitHub\labs-ms\Labs\lab1\Manifest.toml`


In [12]:
using JuMP
using HiGHS

# Datos del problema
costos_produccion = [50, 45, 55, 52, 48, 50]
costos_inventario = [8, 10, 10, 10, 8, 8]
demanda = [180, 250, 190, 140, 220, 250]
capacidad = 225
n = length(demanda)

# Crear el modelo
model = Model(HiGHS.Optimizer)

# Variables: producción x[i] y inventario I[i]
@variable(model, x[1:n] >= 0)     # producción mensual
@variable(model, I[1:n] >= 0)     # inventario mensual

# Restricciones de inventario
@constraint(model, x[1] - demanda[1] == I[1])  # mes 1

for i in 2:n
    @constraint(model, I[i-1] + x[i] - demanda[i] == I[i])
end

# Restricciones de capacidad máxima
for i in 1:n
    @constraint(model, x[i] <= capacidad)
end

# Función objetivo: minimizar costo total
@objective(model, Min, sum(costos_produccion[i]*x[i] + costos_inventario[i]*I[i] for i in 1:n))


50 x[1] + 8 I[1] + 45 x[2] + 10 I[2] + 55 x[3] + 10 I[3] + 52 x[4] + 10 I[4] + 48 x[5] + 8 I[5] + 50 x[6] + 8 I[6]

## Inciso b

Resolver el problema usando la librería JuMP o Pulp, en variables continuas, y determinar la producción óptima. Haga un diagrama o esquema de la producción óptima, similar a la Figura 2, indicando las producciones en cada mes, así como los inventarios. Compare contra la solución (no óptima) que produce exactamente la demanda de cada mes. Elabore una tabla mes a mes, indicando los costos de producción, los costos de inventario, el costo total y el ahorro generado al compararlo con la producción no óptima anterior.

In [ ]:
# Resolver el modelo
optimize!(model)

# Obtener los valores óptimos de producción e inventario
produccion_opt = value.(x)
inventario_opt = value.(I)

# Crear tabla como lista de vectores
tabla = [ 
    [i,
     produccion_opt[i],
     inventario_opt[i],
     costos_produccion[i] * produccion_opt[i],
     costos_inventario[i] * inventario_opt[i]] 
    for i in 1:n
]

# Convertir a matriz (formato esperado por PrettyTables)
tabla_matriz = Matrix(reduce(hcat, tabla)')  # ¡Aquí está la corrección clave!

# Encabezado
encabezado = ["Mes", "Producción", "Inventario", "Costo producción", "Costo inventario"]

# Mostrar la tabla con formato bonito
pretty_table(tabla_matriz; header = encabezado, formatters = (v, i, j) -> @sprintf("%.1f", v))



LP   has 12 rows; 12 cols; 23 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [8e+00, 6e+01]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 2e+02]
Solving LP without presolve, or with basis, or unconstrained
Model status        : Optimal
Objective value     :  6.1795000000e+04
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00
┌─────┬────────────┬────────────┬──────────────────┬──────────────────┐
│ Mes │ Producción │ Inventario │ Costo producción │ Costo inventario │
├─────┼────────────┼────────────┼──────────────────┼──────────────────┤
│ 1.0 │      205.0 │       25.0 │          10250.0 │            200.0 │
│ 2.0 │      225.0 │        0.0 │          10125.0 │              0.0 │
│ 3.0 │      190.0 │        0.0 │          10450.0 │              0.0 │
│ 4.0 │      160.0 │       20.0 │           8320.0 │            200.0 │
│ 5.0 │      225.0 │       25.0 │          10800.0 │            200.0 │
│ 6.0 │      225.0 │        0.0 │          11250.0 │          

## Inciso c

¿Se obtiene la misma solución óptima al introducir restricciones enteras?

In [ ]:
using Printf  # asegúrate de tener esto arriba para usar @sprintf

# Crear nuevo modelo con variables enteras
modelo_entero = Model(HiGHS.Optimizer)

@variable(modelo_entero, x_ent[1:n] >= 0, Int)  # producción entera
@variable(modelo_entero, I_ent[1:n] >= 0)       # inventario continuo

# Restricciones
@constraint(modelo_entero, x_ent[1] - demanda[1] == I_ent[1])
for i in 2:n
    @constraint(modelo_entero, I_ent[i-1] + x_ent[i] - demanda[i] == I_ent[i])
end

for i in 1:n
    @constraint(modelo_entero, x_ent[i] <= capacidad)
end

# Función objetivo
@objective(modelo_entero, Min, sum(costos_produccion[i]*x_ent[i] + costos_inventario[i]*I_ent[i] for i in 1:n))

# Resolver
optimize!(modelo_entero)

# Resultados
produccion_entera = value.(x_ent)
inventario_entero = value.(I_ent)

# Crear tabla como matriz
tabla_entera = [
    [i,
     produccion_entera[i],
     inventario_entero[i],
     costos_produccion[i] * produccion_entera[i],
     costos_inventario[i] * inventario_entero[i]]
    for i in 1:n
]

tabla_matriz_entera = Matrix(reduce(hcat, tabla_entera)')  # convertir a matriz real

# Encabezado de columnas (puedes reutilizar el de antes o volverlo a definir)
encabezado = ["Mes", "Producción", "Inventario", "Costo producción", "Costo inventario"]

# Mostrar tabla
println("\nTabla con variables ENTERAS:")
pretty_table(tabla_matriz_entera; header = encabezado, formatters = (v, i, j) -> @sprintf("%.1f", v))

# Comparación de costos
println("\nCosto total óptimo (con producción entera): \$", round(objective_value(modelo_entero), digits=2))
println("Costo total óptimo anterior (con producción continua): \$", round(objective_value(model), digits=2))

ahorro_ent_vs_cont = objective_value(modelo_entero) - objective_value(model)
println("Diferencia por usar enteros: \$", round(ahorro_ent_vs_cont, digits=2))


Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 12 rows; 12 cols; 23 nonzeros; 6 integer variables (0 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [8e+00, 6e+01]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 2e+02]
Presolving model
5 rows, 11 cols, 15 nonzeros  0s
3 rows, 9 cols, 11 nonzeros  0s
2 rows, 6 cols, 7 nonzeros  0s
2 rows, 5 cols, 6 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   2 rows
   5 cols (0 binary, 3 integer, 2 implied int., 0 continuous, 0 domain fixed)
   6 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |        

MethodError: MethodError: no method matching pretty_table(::Vector{Vector{Float64}}, ::Vector{String}; formatters::PrettyTables.var"#36#39"{Printf.Format{Base.CodeUnits{UInt8, String}, Tuple{Printf.Spec{Val{'f'}}}}})
The function `pretty_table` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  pretty_table(!Matched::Type{HTML}, ::Any; kwargs...)
   @ PrettyTables C:\Users\Cindy\.julia\packages\PrettyTables\oVZqx\src\print.jl:782
  pretty_table(!Matched::Type{String}, ::Any; color, kwargs...)
   @ PrettyTables C:\Users\Cindy\.julia\packages\PrettyTables\oVZqx\src\print.jl:776
  pretty_table(::Any; kwargs...)
   @ PrettyTables C:\Users\Cindy\.julia\packages\PrettyTables\oVZqx\src\print.jl:771
  ...
